In [15]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [16]:
titanic_df = pd.read_csv('data/titanic_processed.csv')
titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0,3,1,15.0,1,1,7.2292,0,0
1,0,3,1,22.0,0,0,7.2500,0,1
2,0,2,1,18.0,0,0,13.0000,0,1
3,1,2,0,18.0,0,1,23.0000,0,1
4,0,3,1,20.0,0,0,7.8542,0,1


In [17]:
features = list(titanic_df.columns[1:])
features

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q', 'Embarked_S']

In [18]:
results_dict = {}

In [19]:
def summarize_classification(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred, normalize = True)
    num_acc = accuracy_score(y_test, y_pred, normalize = False)
    
    prec = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    return {'accuracy' : acc,
            'precision' : prec, 
            'recall' : recall,
            'accurcy_count' : num_acc     
            }

In [20]:
def build_model(classifier_fn, name_of_y_col, names_of_x_cols, dataset, test_frac = 0.2):
    
    x = dataset[names_of_x_cols]
    y = dataset[name_of_y_col]
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_frac)
    
    model = classifier_fn(x_train, y_train)
    y_pred = model.predict(x_test)
    
    y_pred_train = model.predict(x_train)
    
    train_summary = summarize_classification(y_train, y_pred_train)
    test_summary = summarize_classification(y_test, y_pred)
    
    pred_results = pd.DataFrame({'y_test': y_test,
                                 'y_pred': y_pred,
                                })
    
    # This is the confusionmatrix
    model_crosstab = pd.crosstab(pred_results.y_test, pred_results.y_pred)
    
    return {'training' : train_summary,
            'test': test_summary,
            'confusion_matrix' : model_crosstab
           }

In [31]:
def compare_results():
    for key in results_dict:
        print({'classification ' : key})
        
        print()
        print('Training Data')
        for score in results_dict[key]['training']:
            print(score, results_dict[key]['training'][score])
        
        
        print()
        print('Test Data')
        for score in results_dict[key]['test']:
            print(score, results_dict[key]['test'][score])
        
        print()

In [32]:
def logistic_fn(x_train, y_train):
    model = LogisticRegression(solver = 'liblinear')
    model.fit(x_train,y_train)
    
    return model

In [33]:
results_dict['survived - logistic'] = build_model(logistic_fn, 'Survived', features, titanic_df)

compare_results()

{'classification ': 'survived - logistic'}

Training Data
accuracy 0.8031634446397188
precision 0.7804878048780488
recall 0.7048458149779736
accurcy_count 457

Test Data
accuracy 0.7622377622377622
precision 0.7647058823529411
recall 0.639344262295082
accurcy_count 109

{'classification ': 'survived - Linear_Discriminat_Analysis'}

Training Data
accuracy 0.7978910369068541
precision 0.785
recall 0.6855895196506551
accurcy_count 454

Test Data
accuracy 0.8041958041958042
precision 0.746031746031746
recall 0.7966101694915254
accurcy_count 115



In [34]:
def linear_discriminant_fn(x_train, y_train, solver ='svd'):
    model = LinearDiscriminantAnalysis(solver = solver)
    model.fit(x_train, y_train)
    
    return model

In [35]:
results_dict['survived - Linear_Discriminat_Analysis'] = build_model(linear_discriminant_fn, 'Survived', features, titanic_df)

compare_results()

{'classification ': 'survived - logistic'}

Training Data
accuracy 0.8031634446397188
precision 0.7804878048780488
recall 0.7048458149779736
accurcy_count 457

Test Data
accuracy 0.7622377622377622
precision 0.7647058823529411
recall 0.639344262295082
accurcy_count 109

{'classification ': 'survived - Linear_Discriminat_Analysis'}

Training Data
accuracy 0.7908611599297012
precision 0.7605633802816901
recall 0.7043478260869566
accurcy_count 450

Test Data
accuracy 0.8321678321678322
precision 0.8035714285714286
recall 0.7758620689655172
accurcy_count 119



In [36]:
def quadratic_discriminant_fn(x_train, y_train):
    model = QuadraticDiscriminantAnalysis()
    model.fit(x_train, y_train)
    
    return model

In [37]:
results_dict['survived - Quadratic_discriminat_analysis'] = build_model(quadratic_discriminant_fn, 'Survived',features, titanic_df)

compare_results()

{'classification ': 'survived - logistic'}

Training Data
accuracy 0.8031634446397188
precision 0.7804878048780488
recall 0.7048458149779736
accurcy_count 457

Test Data
accuracy 0.7622377622377622
precision 0.7647058823529411
recall 0.639344262295082
accurcy_count 109

{'classification ': 'survived - Linear_Discriminat_Analysis'}

Training Data
accuracy 0.7908611599297012
precision 0.7605633802816901
recall 0.7043478260869566
accurcy_count 450

Test Data
accuracy 0.8321678321678322
precision 0.8035714285714286
recall 0.7758620689655172
accurcy_count 119

{'classification ': 'survived - Quadratic_discriminat_analysis'}

Training Data
accuracy 0.7926186291739895
precision 0.7757009345794392
recall 0.7033898305084746
accurcy_count 451

Test Data
accuracy 0.8111888111888111
precision 0.711864406779661
recall 0.8076923076923077
accurcy_count 116



In [40]:
def sgd_fn(x_train, y_train, max_iter = 1000, tol = 1e-3):
    model = SGDClassifier(max_iter = max_iter, tol = tol)
    model.fit(x_train, y_train)
    
    return model

In [41]:
results_dict['survived - Stochastic_Gradient_Descent'] = build_model(sgd_fn, 'Survived', features, titanic_df)

compare_results()

{'classification ': 'survived - logistic'}

Training Data
accuracy 0.8031634446397188
precision 0.7804878048780488
recall 0.7048458149779736
accurcy_count 457

Test Data
accuracy 0.7622377622377622
precision 0.7647058823529411
recall 0.639344262295082
accurcy_count 109

{'classification ': 'survived - Linear_Discriminat_Analysis'}

Training Data
accuracy 0.7908611599297012
precision 0.7605633802816901
recall 0.7043478260869566
accurcy_count 450

Test Data
accuracy 0.8321678321678322
precision 0.8035714285714286
recall 0.7758620689655172
accurcy_count 119

{'classification ': 'survived - Quadratic_discriminat_analysis'}

Training Data
accuracy 0.7926186291739895
precision 0.7757009345794392
recall 0.7033898305084746
accurcy_count 451

Test Data
accuracy 0.8111888111888111
precision 0.711864406779661
recall 0.8076923076923077
accurcy_count 116

{'classification ': 'survived - Standard_Gradient_Descent'}

Training Data
accuracy 0.7715289982425307
precision 0.69140625
recall 0.776315789473